Parameters Definition

In [1]:
# ------------------ Folders ------------------

# Dataset
DATASET = "dataset/"

# Raw Data
RAW_DATASET = DATASET + "raw/"

# Upsampled Data (for HR and EDA)
UPSAMPLED_DATASET = DATASET + "upsampled/"

# Splitted Data
SPLITTED_DATASET = DATASET + "splitted/"
# Training and Test Sets
TRAINING_SET = SPLITTED_DATASET + "training_set/"
TEST_SET = SPLITTED_DATASET + "test_set/"

# ---------------------------------------------


# ------------------- Files -------------------

# Raw Data
RAW_EEG = RAW_DATASET + "eeg_data.csv"
RAW_HR = RAW_DATASET + "hr_data.csv"
RAW_EDA = RAW_DATASET + "eda_data.csv"

# Upsampled Data
UPSAMPLED_HR = UPSAMPLED_DATASET + "upsampled_hr_data.csv"
UPSAMPLED_EDA = UPSAMPLED_DATASET + "upsampled_eda_data.csv"


# Training and Test Sets
# EEG
EEG_TRAINING_SET = TRAINING_SET + "eeg_training_set.csv"
EEG_TEST_SET = TEST_SET + "eeg_test_set.csv"
# HR
HR_TRAINING_SET = TRAINING_SET + "hr_training_set.csv"
HR_TEST_SET = TEST_SET + "hr_test_set.csv"
# EDA
EDA_TRAINING_SET = TRAINING_SET + "eda_training_set.csv"
EDA_TEST_SET = TEST_SET + "eda_test_set.csv"

# TensorFlowLite Model
TFLITE_MODEL = "eegnet_preproc.tflite"

# ---------------------------------------------


# ----------------- Constants -----------------

# Epoch Duration in seconds
EPOCH_DURATION = 2

# EEG Channels
EEG_CHANNELS = ['ch1','ch2','ch3','ch4','ch5','ch6']
# EEG Sampling Frequency
EEG_SAMPLING_FREQUENCY = 500 # Hz
# Samples per epoch
EEG_SAMPLES_PER_EPOCH = EPOCH_DURATION * EEG_SAMPLING_FREQUENCY  # 2 seconds at 500 Hz

# Target Sampling Frequency
FS_TARGET  = 125 # Hz

# Wearable sampling frequency
WEARABLE_SAMPLES_PER_EPOCH = EPOCH_DURATION * FS_TARGET  # 2 seconds at 125 Hz

# ---------------------------------------------


In [2]:
# ------------------- Imports -------------------
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from scipy.signal import firwin
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Lambda, Conv2D, DepthwiseConv2D, SeparableConv2D,
    BatchNormalization, Activation, AveragePooling2D,
    Dropout, Flatten, Dense, Concatenate
)
from tensorflow.keras.constraints import max_norm
# --------------------------------------------------

Step 1: Upsample data coming from the smartwatch to bring it to 125Hz

In [3]:
# Utility function to upsample data
def upsample_data(group, target_freq=125, duration_s=10):
    # Calculate start and end timestamps per experiment
    start = group['timestamp'].min()
    end = start + duration_s * 1000
    # Generate target timestamps (125 samples/sec × duration)
    target_times = np.linspace(start, end, target_freq * duration_s)
    # Interpolate 'value' at these timestamps
    values = np.interp(target_times, group['timestamp'], group['value'])
    # Build upsampled DataFrame
    up_df = pd.DataFrame({
        'experiment': group['experiment'].iloc[0],
        'timestamp': target_times.astype(int),
        'value': values,
        'subject': group['subject'].iloc[0],
        'rating': group['rating'].iloc[0]
    })
    # Ensure exactly target_freq*duration_s rows
    return up_df.iloc[:target_freq * duration_s]

In [4]:
# Heart Rate

# 1. Read the raw CSV (all experiments)
raw_df = pd.read_csv(RAW_HR)

# 2. Upsample each experiment
upsampled_df = raw_df.groupby('experiment', group_keys=False).apply(upsample_data)

# 3. Export to CSV

# Creating output directory if it doesn't exist
os.makedirs(UPSAMPLED_DATASET, exist_ok=True)

upsampled_df.to_csv(UPSAMPLED_HR, index=False)

print(f"Upsampling complete! Data saved to {UPSAMPLED_HR}.")

Upsampling complete! Data saved to dataset/upsampled/upsampled_hr_data.csv.


C:\Users\giova\AppData\Local\Temp\ipykernel_20216\2837762272.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  upsampled_df = raw_df.groupby('experiment', group_keys=False).apply(upsample_data)


In [5]:
# EDA

# 1. Read the raw CSV (all experiments)
raw_df = pd.read_csv(RAW_EDA)

# 2. Upsample each experiment
upsampled_df = raw_df.groupby('experiment', group_keys=False).apply(upsample_data)

# 3. Export to CSV

# Creating output directory if it doesn't exist
os.makedirs(UPSAMPLED_DATASET, exist_ok=True)

upsampled_df.to_csv(UPSAMPLED_EDA, index=False)

print(f"Upsampling complete! Data saved to {UPSAMPLED_EDA}.")

C:\Users\giova\AppData\Local\Temp\ipykernel_20216\2060052639.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  upsampled_df = raw_df.groupby('experiment', group_keys=False).apply(upsample_data)


Upsampling complete! Data saved to dataset/upsampled/upsampled_eda_data.csv.


Step 2: Split data into training and test sets using an hold-out strategy

In [6]:
# Load the EEG raw dataset
eeg_df = pd.read_csv(RAW_EEG)

# Load the upsampled heart-rate dataset
hr_df = pd.read_csv(UPSAMPLED_HR)

# Load the Upsampled eda dataset
eda_df = pd.read_csv(UPSAMPLED_EDA)

# Extract unique experiment IDs from the EEG data
experiments = eeg_df['experiment'].unique()

# Split experiments: 80% train, 20% test
train_exps, test_exps = train_test_split(
    experiments,
    test_size=0.2,
    random_state=42
)

# Partition the three datasets by the same experiment split
# EEG splits
eeg_train = eeg_df[eeg_df['experiment'].isin(train_exps)].reset_index(drop=True)
eeg_test  = eeg_df[eeg_df['experiment'].isin(test_exps)].reset_index(drop=True)

# Heart-rate splits
hr_train = hr_df[hr_df['experiment'].isin(train_exps)].reset_index(drop=True)
hr_test  = hr_df[hr_df['experiment'].isin(test_exps)].reset_index(drop=True)

# EDA splits
eda_train = eda_df[eda_df['experiment'].isin(train_exps)].reset_index(drop=True)
eda_test  = eda_df[eda_df['experiment'].isin(test_exps)].reset_index(drop=True)

# Creating the folders if they do not exist
# Create the training set folder
if not os.path.exists(TRAINING_SET):
    os.makedirs(TRAINING_SET)

# Create the test set folder
if not os.path.exists(TEST_SET):
    os.makedirs(TEST_SET)

# Save the splitted files
# EEG
eeg_train.to_csv(EEG_TRAINING_SET, index=False)
eeg_test.to_csv(EEG_TEST_SET, index=False)

# Heart-rate
hr_train.to_csv(HR_TRAINING_SET, index=False)
hr_test.to_csv(HR_TEST_SET, index=False)

# EDA
eda_train.to_csv(EDA_TRAINING_SET, index=False)
eda_test.to_csv(EDA_TEST_SET, index=False)

# Sanity checks
print("Train experiments:", train_exps)
print("Test experiments :", test_exps)
print(f"EEG train rows: {len(eeg_train)}, test rows: {len(eeg_test)}")
print(f"HR train rows: {len(hr_train)}, test rows: {len(hr_test)}")
print(f"EDA train rows: {len(eda_train)}, test rows: {len(eda_test)}")

Train experiments: [ 4  7 25 33 20 18 14 43 16 10 17 38 32 28  1 31 30  6 12 34  2 41 22  3
 35 24 37 11 23 19 45 21  8 15 29 39]
Test experiments : [40 26 27 44 36 42  5 13  9]
EEG train rows: 180000, test rows: 45000
HR train rows: 45000, test rows: 11250
EDA train rows: 45000, test rows: 11250


Step 3: Model definition

In [7]:
# -------------------- Design FIR filters --------------------
fs_in       = 500                   # incoming sample rate
fs_target   = 125                   # desired rate after decimation
dec_factor  = fs_in // fs_target    # must be 4
numtaps     = 101                   # filter length 

# 1) High-pass (0.5 Hz) for detrend
hp_coeffs   = firwin(numtaps, cutoff=0.5, fs=fs_in, pass_zero=False)
# 2) Band-pass (0.5–50 Hz)
bp_coeffs   = firwin(numtaps, cutoff=[0.5, 50.], fs=fs_in, pass_zero=False)
# 3) Notch (49–51 Hz)
bs_coeffs   = firwin(numtaps, cutoff=[49., 51.], fs=fs_in, pass_zero=True)
# 4) Anti-alias low-pass (≤62.5 Hz)
aa_coeffs   = firwin(numtaps, cutoff=fs_target/2, fs=fs_in)

# -------------------- Helper to apply 1D FIR --------------------
def apply_fir(x, coeffs):
    """ x: (B, Ch, T, 1) → applies 1-D FIR along time via depthwise conv """
    # 1) remove trailing singleton and transpose to NHWC with time in W, channels in C:
    #    (B, Ch, T, 1) → (B, T, Ch)
    x2 = tf.squeeze(x, -1)
    #    (B, T, Ch) → (B, 1, T, Ch)
    x2 = tf.expand_dims(x2, 1)

    # 2) build depthwise kernel of shape (1, filter_len, in_channels=Ch, channel_multiplier=1)
    #    coeffs is (filter_len,), so first reshape to (1, filter_len, 1, 1)
    k = tf.constant(coeffs.reshape(1, -1, 1, 1), tf.float32)
    #    then tile the "in_channels" dimension to match your EEG channels
    n_ch = x2.shape[-1]
    k = tf.tile(k, [1, 1, n_ch, 1])  # now (1, filter_len, Ch, 1)

    # 3) depthwise conv2d → same padding
    y = tf.nn.depthwise_conv2d(
        x2,
        k,
        strides=[1, 1, 1, 1],
        padding='SAME',
        data_format='NHWC'
    )
    # y has shape (B, 1, T, Ch); we want back (B, Ch, T, 1)

    y = tf.squeeze(y, 1)            # → (B, T, Ch)
    y = tf.transpose(y, [0, 2, 1])  # → (B, Ch, T)
    return y[..., tf.newaxis]       # → (B, Ch, T, 1)


# -------------------- Combined model --------------------
def PreprocAndEEGNet(nb_classes,
                     channel_means, channel_stds,  # arrays of shape (8,)
                     eeg_chans=6,
                     fs_in=500,                    # original raw rate for EEG
                     fs_target=125,                # rate after decimation
                     duration_s=2.0,               # length of each input epoch in seconds
                     dropoutRate=0.5,
                     F1=8, D=2, F2=16,
                     norm_rate=0.25):
    # Calculate dynamic shapes
    Samples_in = int(fs_in * duration_s)       # e.g. 1000 samples @ 500 Hz
    Samples    = int(fs_target * duration_s)   # e.g. 250 samples @ 125 Hz
    dec_factor = fs_in // fs_target            # 4
    kernLength = int(0.5 * fs_target)          # half the target rate: 62
    
    # EEG input path
    eeg_in = Input(shape=(eeg_chans, Samples_in, 1), name='eeg_input')

    # 1) High-pass
    x = Lambda(lambda z: apply_fir(z, hp_coeffs), output_shape=lambda input_shape: input_shape, name='hp')(eeg_in)
    # 2) Band-pass
    x = Lambda(lambda z: apply_fir(z, bp_coeffs), output_shape=lambda input_shape: input_shape, name='bp')(x)
    # 3) Notch
    x = Lambda(lambda z: apply_fir(z, bs_coeffs), output_shape=lambda input_shape: input_shape, name='notch')(x)
    # 4) Anti-alias low-pass
    x = Lambda(lambda z: apply_fir(z, aa_coeffs), output_shape=lambda input_shape: input_shape, name='aa')(x)
    # 5) Decimate by slicing every dec_factor-th sample
    x = Lambda(lambda z: z[:, :, ::dec_factor, :], name='decimate')(x)

    # now x has shape (batch, Chans, Samples_in/4, 1) → (batch,6,250,1)

    # HR input path (no filtering, already 125 Hz)
    hr_in = Input(shape=(1, Samples, 1), name='hr_input')

    # EDA input path (no filtering, already 125 Hz)
    eda_in = Input(shape=(1, Samples, 1), name='eda_input')
    
    # 3) Concatenate along channel axis → (B, 8, 250, 1)
    merged = Concatenate(axis=1)([x, hr_in, eda_in])
    Chans = eeg_chans + 2  # now 8
    
    # --- EEGNet layers expects Samples=250 ---
    # Keras Implementation of EEGNet
    # http://iopscience.iop.org/article/10.1088/1741-2552/aace8c/meta

    # Normalization
    # we broadcast: (1,8,1,1) → will divide/subtract per-channel
    means = tf.constant(channel_means.reshape(1, -1, 1, 1), dtype=tf.float32)
    stds  = tf.constant(channel_stds .reshape(1, -1, 1, 1), dtype=tf.float32)
    norm  = Lambda(lambda z: (z - means) / stds, name="normalize")(merged)

    # Block 1
    b1 = Conv2D(F1, (1, kernLength), padding='same', use_bias=False)(norm)
    b1 = BatchNormalization()(b1)
    b1 = DepthwiseConv2D((Chans, 1), use_bias=False,
                        depth_multiplier=D,
                        depthwise_constraint=max_norm(1.))(b1)
    b1 = BatchNormalization()(b1)
    b1 = Activation('elu')(b1)
    b1 = AveragePooling2D((1, 4))(b1)
    b1 = Dropout(dropoutRate)(b1)

    # Block 2
    b2 = SeparableConv2D(F2, (1, 16), padding='same', use_bias=False)(b1)
    b2 = BatchNormalization()(b2)
    b2 = Activation('elu')(b2)
    b2 = AveragePooling2D((1, 8))(b2)
    b2 = Dropout(dropoutRate)(b2)
    
    # Classification head
    flat = Flatten(name='flatten')(b2)
    dense= Dense(nb_classes, kernel_constraint=max_norm(norm_rate), name='dense')(flat)
    out  = Activation('softmax', name='softmax')(dense)
        
    return Model(inputs=[eeg_in, hr_in, eda_in], outputs=out, name='EEGNet_with_Preproc')
    

# Example instantiation
model = PreprocAndEEGNet(nb_classes=5, channel_means=np.zeros(8), channel_stds=np.ones(8))

# Compile / train
model.compile('adam', 'sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "EEGNet_with_Preproc"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ eeg_input           │ (None, 6, 1000,   │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ hp (Lambda)         │ (None, 6, 1000,   │          0 │ eeg_input[0][0]   │
│                     │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bp (Lambda)         │ (None, 6, 1000,   │          0 │ hp[0][0]          │
│                     │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ notch (Lambda)      │ (None, 6, 1000,   │          0 │ bp[0][0]          │
│                     │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ aa (Lambda)         │ (None, 6, 1000,   │          0 │ notch[0][0]       │
│                     │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decimate (Lambda)   │ (None, 6, 250, 1) │          0 │ aa[0][0]          │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ hr_input            │ (None, 1, 250, 1) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ eda_input           │ (None, 1, 250, 1) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 8, 250, 1) │          0 │ decimate[0][0],   │
│ (Concatenate)       │                   │            │ hr_input[0][0],   │
│                     │                   │            │ eda_input[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalize (Lambda)  │ (None, 8, 250, 1) │          0 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 8, 250, 8) │        496 │ normalize[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 8, 250, 8) │         32 │ conv2d[0][0]      │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ depthwise_conv2d    │ (None, 1, 250,    │        128 │ batch_normalizat… │
│ (DepthwiseConv2D)   │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 1, 250,    │         64 │ depthwise_conv2d… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 1, 250,    │          0 │ batch_normalizat… │
│ (Activation)        │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ average_pooling2d   │ (None, 1, 62, 16) │          0 │ activation[0][0]  │
│ (AveragePooling2D)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 1, 62, 16) │          0 │ average_pooling2… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ separable_conv2d    │ (None, 1, 62, 16) │        512 │ dropout[0][0]   

 Total params: 1,861 (7.27 KB)

 Trainable params: 1,781 (6.96 KB)

 Non-trainable params: 80 (320.00 B)

Step 4: Model training and evaluation

In [8]:
# Utility functions for loading epochs

def load_eeg_epochs(csv_file, channels, packets_per_epoch):
    df = pd.read_csv(csv_file)
    df['epoch_idx'] = (df['sample'] // packets_per_epoch).astype(int)
    X_list, y_list = [], []
    for (_, _, epoch_idx), grp in df.groupby(['experiment','subject','epoch_idx']):
        grp = grp.sort_values('sample')
        data = grp[channels].values.T  # (6, packets_per_epoch)
        X_list.append(data[..., np.newaxis])
        y_list.append(grp['rating'].iloc[0])
    return np.stack(X_list), np.array(y_list)

def load_wearable_epochs(csv_file, samples_per_epoch):
    """
    Load and segment HR or EDA data into non-overlapping epochs based on row index.
    
    Parameters:
    - csv_file: path to CSV with columns ['experiment','timestamp','value','subject','rating']
    - samples_per_epoch: number of consecutive samples per epoch (e.g. 2 s * 125 Hz = 250)
    
    Returns:
    - X_eda: np.ndarray of shape (n_epochs, 1, samples_per_epoch, 1)
    - y_eda: np.ndarray of labels (n_epochs,)
    """
    df = pd.read_csv(csv_file)
    # Ensure consistent ordering
    df = df.reset_index(drop=True)
    # Epoch index based on row index
    df['epoch_idx'] = df.index // samples_per_epoch

    X_list = []
    y_list = []

    # Group by experiment, subject, and computed epoch index
    for (exp, subj, ep), grp in df.groupby(['experiment', 'subject', 'epoch_idx']):
        grp = grp.sort_index()
        values = grp['value'].values
        if len(values) == samples_per_epoch:
            # shape for each epoch: (1 channel, samples, 1)
            X_list.append(values[np.newaxis, :, np.newaxis])
            y_list.append(grp['rating'].iloc[0])

    X_eda = np.concatenate(X_list, axis=0)
    y_eda = np.array(y_list)
    return X_eda, y_eda

In [9]:
# Load the training data
X_eeg_train, y_train = load_eeg_epochs(EEG_TRAINING_SET, EEG_CHANNELS, EEG_SAMPLES_PER_EPOCH)
X_hr_train, y_hr_train = load_wearable_epochs(HR_TRAINING_SET, WEARABLE_SAMPLES_PER_EPOCH)
X_eda_train, y_eda_train = load_wearable_epochs(EDA_TRAINING_SET, WEARABLE_SAMPLES_PER_EPOCH)

# Assert that y_train, y_hr_train, and y_eda_train are the same
assert np.array_equal(y_train, y_hr_train), "y_train and y_hr_train do not match!"
assert np.array_equal(y_train, y_eda_train), "y_train and y_eda_train do not match!"

# Shuffle the data
X_eeg_train, X_hr_train, X_eda_train, y_train = shuffle(X_eeg_train, X_hr_train, X_eda_train, y_train, random_state=42)

In [10]:
# Precompute preprocessed EEG (to get means/stds)
# Build a small model that inputs raw EEG and outputs the decimated, filtered EEG:
eeg_in = tf.keras.Input(shape=(6, EEG_SAMPLES_PER_EPOCH, 1))
x = tf.keras.layers.Lambda(lambda z: apply_fir(z, hp_coeffs))(eeg_in)
x = tf.keras.layers.Lambda(lambda z: apply_fir(z, bp_coeffs))(x)
x = tf.keras.layers.Lambda(lambda z: apply_fir(z, bs_coeffs))(x)
x = tf.keras.layers.Lambda(lambda z: apply_fir(z, aa_coeffs))(x)
x = tf.keras.layers.Lambda(lambda z: z[:, :, ::4, :])(x)
preproc_model = tf.keras.Model(inputs=eeg_in, outputs=x, name='eeg_preproc')

# Run preprocessing on the entire training set
X_eeg_pre = preproc_model.predict(X_eeg_train, batch_size=16)  # shape (n_train,6,250,1)

# Align shapes and concatenate EEG+HR+EDA
# X_hr_train shape: (n_train, 250, 1) → reshape to (n_train, 1, 250, 1)
X_hr_train = X_hr_train.reshape(-1, 1, WEARABLE_SAMPLES_PER_EPOCH, 1)
# X_eda_train shape: (n_train, 250, 1) → reshape to (n_train, 1, 250, 1)
X_eda_train = X_eda_train.reshape(-1, 1, WEARABLE_SAMPLES_PER_EPOCH, 1)

X_merged = np.concatenate([X_eeg_pre, X_hr_train, X_eda_train], axis=1)  # shape (n_train,8,250,1)

# Compute channel-wise mean/std
channel_means = X_merged.mean(axis=(0,2,3))  # shape (8,)
channel_stds  = X_merged.std (axis=(0,2,3))  # shape (8,)

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


In [11]:
# Build and compile the model
model = PreprocAndEEGNet(
    nb_classes=5,
    channel_means=channel_means,
    channel_stds=channel_stds
)
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    {'eeg_input': X_eeg_train, 'hr_input': X_hr_train, 'eda_input': X_eda_train},
    y_train,
    batch_size=128,
    epochs=1000,
    validation_split=0.1,
    verbose=1
)

Epoch 1/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 377ms/step - accuracy: 0.2479 - loss: 1.8569 - val_accuracy: 0.3889 - val_loss: 1.6098
Epoch 2/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - accuracy: 0.2558 - loss: 1.5882 - val_accuracy: 0.3333 - val_loss: 1.6088
Epoch 3/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step - accuracy: 0.2547 - loss: 1.5738 - val_accuracy: 0.2778 - val_loss: 1.6079
Epoch 4/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - accuracy: 0.3544 - loss: 1.5440 - val_accuracy: 0.2778 - val_loss: 1.6069
Epoch 5/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step - accuracy: 0.3316 - loss: 1.5508 - val_accuracy: 0.2778 - val_loss: 1.6061
Epoch 6/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step - accuracy: 0.3869 - loss: 1.5331 - val_accuracy: 0.2778 - val_loss: 1.6054
Epoch 7/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step - accuracy: 0.3383 - loss: 1.5230 - val_accuracy: 0.2778 - val_loss: 1.6047
Epoch 8/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - accuracy: 0.3906 - loss: 1.4977 - val_accuracy: 0.2778 

In [12]:
# Evaluate the model

# Load the test data
X_eeg_test, y_test = load_eeg_epochs(EEG_TEST_SET, EEG_CHANNELS, EEG_SAMPLES_PER_EPOCH)
X_hr_test, y_hr_test = load_wearable_epochs(HR_TEST_SET, WEARABLE_SAMPLES_PER_EPOCH)
X_eda_test, y_eda_test = load_wearable_epochs(EDA_TEST_SET, WEARABLE_SAMPLES_PER_EPOCH)

# Assert that y_test, y_hr_test, and y_eda_test are the same
assert np.array_equal(y_test, y_hr_test), "y_test and y_hr_test do not match!"
assert np.array_equal(y_test, y_eda_test), "y_test and y_eda_test do not match!"

# Shuffle the data
X_eeg_test, X_hr_test, X_eda_test, y_test = shuffle(X_eeg_test, X_hr_test, X_eda_test, y_test, random_state=42)

# Align shapes
# X_hr_test shape: (n_test, 250, 1) → reshape to (n_test, 1, 250, 1)
X_hr_test = X_hr_test.reshape(-1, 1, WEARABLE_SAMPLES_PER_EPOCH, 1)
# X_eda_test shape: (n_test, 250, 1) → reshape to (n_test, 1, 250, 1)
X_eda_test = X_eda_test.reshape(-1, 1, WEARABLE_SAMPLES_PER_EPOCH, 1)

# Model evaluation
test_loss, test_accuracy = model.evaluate(
    {'eeg_input': X_eeg_test, 'hr_input': X_hr_test, 'eda_input': X_eda_test},
    y_test,
    batch_size=128,
    verbose=1
)
print(f"Test loss: {test_loss:.4f}, Test accuracy: {test_accuracy:.4f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - accuracy: 0.3333 - loss: 2.3257
Test loss: 2.3257, Test accuracy: 0.3333


Step 5: Model conversion to TFLite

In [13]:
# -------------------- Conversion to TFLite --------------------
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]  # optional quantization
tflite_model = converter.convert()
with open(TFLITE_MODEL,'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\giova\AppData\Local\Temp\tmp2j3zio5f\assets


INFO:tensorflow:Assets written to: C:\Users\giova\AppData\Local\Temp\tmp2j3zio5f\assets


Saved artifact at 'C:\Users\giova\AppData\Local\Temp\tmp2j3zio5f'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): List[TensorSpec(shape=(None, 6, 1000, 1), dtype=tf.float32, name='eeg_input'), TensorSpec(shape=(None, 1, 250, 1), dtype=tf.float32, name='hr_input'), TensorSpec(shape=(None, 1, 250, 1), dtype=tf.float32, name='eda_input')]
Output Type:
  TensorSpec(shape=(None, 5), dtype=tf.float32, name=None)
Captures:
  1586326941264: TensorSpec(shape=(1, 8, 1, 1), dtype=tf.float32, name=None)
  1585172267952: TensorSpec(shape=(1, 8, 1, 1), dtype=tf.float32, name=None)
  1585172282208: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1585172564624: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1585172563744: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1585172279216: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1585172563392: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1585172272704: TensorSpec(shape=(), dtype=tf